In [8]:
#dependencies
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize, StandardScaler

In [9]:
# read csv
user_artist_data = pd.read_csv("Resources/user_artists.csv")
artists_df = pd.read_csv('Resources/artists.csv')
artists_df = artists_df.rename(columns = {"id":"artistID"})
artists_df

,artistID,name,url,pictureURL,Unnamed: 4,Unnamed: 5
0,1,MALICE MIZER,http://www.last.fm/music/MALICE+MIZER,http://userserve-ak.last.fm/serve/252/10808.jpg,NaN,NaN
1,2,Diary of Dreams,http://www.last.fm/music/Diary+of+Dreams,http://userserve-ak.last.fm/serve/252/3052066.jpg,NaN,NaN
2,3,Carpathian Forest,http://www.last.fm/music/Carpathian+Forest,http://userserve-ak.last.fm/serve/252/40222717...,NaN,NaN
3,4,Moi dix Mois,http://www.last.fm/music/Moi+dix+Mois,http://userserve-ak.last.fm/serve/252/54697835...,NaN,NaN
4,5,Bella Morte,http://www.last.fm/music/Bella+Morte,http://userserve-ak.last.fm/serve/252/14789013...,NaN,NaN
...,...,...,...,...,...,...
17627,18741,Diamanda Galás,http://www.last.fm/music/Diamanda+Gal%C3%A1s,http://userserve-ak.last.fm/serve/252/16352971...,NaN,NaN
17628,18742,Aya RL,http://www.last.fm/music/Aya+RL,http://userserve-ak.last.fm/serve/252/207445.jpg,NaN,NaN
17629,18743,Coptic Rain,http://www.last.fm/music/Coptic+Rain,http://userserve-ak.last.fm/serve/252/344868.jpg,NaN,NaN
17630,18744,Oz Alchemist,http://www.last.fm/music/Oz+Alchemist,http://userserve-ak.last.fm/serve/252/29297695...,NaN,NaN


In [11]:
user_tag_df = pd.read_csv('Resources/user_taggedartists.csv')
user_tag_df.head()

,userID,artistID,tagID,day,month,year
0,2,52,13,1,4,2009
1,2,52,15,1,4,2009
2,2,52,18,1,4,2009
3,2,52,21,1,4,2009
4,2,52,41,1,4,2009


In [12]:
user_artist_data.head()

,userID,artistID,weight
0,2,51,13883
1,2,52,11690
2,2,53,11351
3,2,54,10300
4,2,55,8983


In [13]:
# Merge dataframes 
df = pd.merge(user_artist_data,artists_df, on = 'artistID', how='left')
df

,userID,artistID,weight,name,url,pictureURL,Unnamed: 4,Unnamed: 5
0,2,51,13883,Duran Duran,http://www.last.fm/music/Duran+Duran,http://userserve-ak.last.fm/serve/252/155668.jpg,NaN,NaN
1,2,52,11690,Morcheeba,http://www.last.fm/music/Morcheeba,http://userserve-ak.last.fm/serve/252/46005111...,NaN,NaN
2,2,53,11351,Air,http://www.last.fm/music/Air,http://userserve-ak.last.fm/serve/252/251119.jpg,NaN,NaN
3,2,54,10300,Hooverphonic,http://www.last.fm/music/Hooverphonic,http://userserve-ak.last.fm/serve/252/40553471...,NaN,NaN
4,2,55,8983,Kylie Minogue,http://www.last.fm/music/Kylie+Minogue,http://userserve-ak.last.fm/serve/252/12740835...,NaN,NaN
...,...,...,...,...,...,...,...,...
92829,2100,18726,337,Nyktalgia,http://www.last.fm/music/Nyktalgia,http://userserve-ak.last.fm/serve/252/49060167...,NaN,NaN
92830,2100,18727,297,Atsakau niekadA,http://www.last.fm/music/Atsakau++niekadA,http://userserve-ak.last.fm/serve/252/29862435...,NaN,NaN
92831,2100,18728,281,Domantas Razauskas,http://www.last.fm/music/Domantas+Razauskas,http://userserve-ak.last.fm/serve/252/165556.jpg,NaN,NaN
92832,2100,18729,280,Atalyja,http://www.last.fm/music/Atalyja,http://userserve-ak.last.fm/serve/252/98093.jpg,NaN,NaN


In [14]:
df2= df.drop(['url','pictureURL','Unnamed: 4', 'Unnamed: 5' ], axis=1)
df2

,userID,artistID,weight,name
0,2,51,13883,Duran Duran
1,2,52,11690,Morcheeba
2,2,53,11351,Air
3,2,54,10300,Hooverphonic
4,2,55,8983,Kylie Minogue
...,...,...,...,...
92829,2100,18726,337,Nyktalgia
92830,2100,18727,297,Atsakau niekadA
92831,2100,18728,281,Domantas Razauskas
92832,2100,18729,280,Atalyja


In [22]:
duplicate_rows = df2[df2.duplicated()]

# Print duplicate rows if any
if not duplicate_rows.empty:
    print("Duplicate Rows:")
    print(duplicate_rows)
else:
    print("No duplicate rows found.")

No duplicate rows found.


In [16]:
user_artist_data['userID'].value_counts().tail(50)

995     38
712     38
1665    37
1859    37
1868    36
344     35
613     34
942     34
493     33
769     33
1290    32
486     32
192     30
845     28
1543    27
1014    25
1810    22
1262    21
408     18
35      15
1744    15
1205    14
270     14
1397    14
1675    13
1552    12
558     12
701     11
797     11
745     10
869     10
1773    10
1432     9
542      7
1115     5
1334     4
1634     4
374      4
1893     4
1611     4
1266     3
188      2
1603     1
1307     1
112      1
2085     1
1758     1
615      1
1013     1
1731     1
Name: userID, dtype: int64

In [24]:
user_artist_data['artistID'].value_counts().head(50)

89      611
289     522
288     484
227     480
300     473
67      429
333     417
292     407
190     400
498     399
295     397
154     393
65      369
466     362
701     319
302     305
306     304
229     304
55      298
461     286
72      282
377     280
157     266
291     260
234     258
163     258
679     249
207     248
344     247
298     243
173     242
257     241
230     236
228     235
349     233
159     229
378     226
707     224
220     222
486     216
533     210
299     209
959     208
475     204
599     198
1412    196
903     195
511     185
424     184
325     182
Name: artistID, dtype: int64